In [ ]:
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 


from Artha.nlp_extraction import *
import Artha.data_process as dp
from Artha.neocoinsdb import Neo
import Artha.crypto_data as crypto
from tqdm import tqdm
import csv
import json
from collections import Counter
import numpy as np
np.set_printoptions(suppress=True)
import pprint
import math
from datetime import datetime

pp = pprint.PrettyPrinter(indent=1)

# coins database
neo = Neo("neo4j://localhost:7687", "neo4j", "part2")

In [ ]:
print(len(neo.get_nodes()), len(neo.get_relations()))

In [ ]:
neo.clear_db()

In [ ]:
neo.load_coins_data()

In [ ]:
%%time
markets = crypto.get_market_dict()
inv_markets = crypto.get_invert_dict(markets)

In [ ]:
%%time
# username = "BTC_JackSparrow"
# username = "razoreth"
# username = "Nostranomist"
username = "CryptoKaleo"
docs = run_pipeline(username)
all_ticks = Counter([tick for doc in docs for tick in doc._.tickers ]).most_common()
just_ticks = [i[0] for i in all_ticks]

In [ ]:
%%time
# construct empty dicts
def time_diff(date_string):
    time = datetime.now() - datetime.strptime(date_string, '%m/%d/%Y %H:%M:%S')
    return time.days + (time.seconds+time.microseconds/(10**6))/86400
def expo_func(x):
    return np.exp(-.08*x)



tick_dict = dict(zip(just_ticks, np.arange(len(just_ticks))))

mentions = np.zeros((len(tick_dict), len(docs)))

# fill in values of dict lists where ticker is mentioned
for ind, doc in tqdm(enumerate(docs)):
    for tick in doc._.tickers:
        mentions[tick_dict[tick]][ind] += 1

# function of array of tweet times
tweet_times = expo_func(np.array([time_diff(doc._.tweeted_at) for doc in docs]).reshape((len(docs),1)))

# multiply matrix of mentions by time vector
mul = np.matmul(mentions, tweet_times).round(5).reshape(1, len(all_ticks))[0]